In [4]:
import simpy
import random

### Task: 

[https://drive.google.com/file/d/1JwpS2P06_k_mz9l496XZz9cmscQSMGdW/view?usp=sharing]


### Case Study - Simulating a container


In [5]:
SIMULATION_TIME = 24 * 60  # simulation time (in mins)
VESSEL_AVG_ARRIVAL_INTERVAL = 5 * 60  # vessels average arrival time (in mins)
CONTAINERS_PER_VESSEL = 150  # no of containers per vessel
AVAIL_BERTHS = 2  # no of berths
AVAIL_CRANES = 2  # no of cranes in service
AVAIL_TRUCKS = 3  # no of trucks in service
MOVE_CONTAINER_TIME = 3  # time elapsed by crane to move container to the truck (in mins)
TRUCK_TRIP_ROUND_TIME = 6  # time taken by trucks to move container to the yard (in mins)

In [26]:
class ContainerSimulation:
    
    def __init__(self, env, berths, cranes, trucks):
        self.env = env
        self.berths = simpy.Resource(env, berths)
        self.cranes = simpy.Resource(env, cranes)
        self.trucks = simpy.Resource(env, trucks)

    def process_for_unloading_containers(self, vessel_id):
        print(f"Vessel {vessel_id} arriving at time {self.env.now:.2f}")

        berth_request = self.berths.request()
        yield berth_request
        print(f"Vessel {vessel_id} berthing at time {self.env.now:.2f}")
        
        for container_no in range(CONTAINERS_PER_VESSEL):
                crane_req = self.cranes.request()
                yield crane_req
                
                yield self.env.process(self.Process_for_loading_container_on_truck(vessel_id, container_no+1))
                self.cranes.release(crane_req)
        self.berths.release(berth_request)
        print(f"Vessel {vessel_id} leaving at time {self.env.now:.2f}")

    def Process_for_loading_container_on_truck(self, vessel_id, container_no):
        trucks_req = self.trucks.request()
        yield trucks_req
    
        print(f"Quay crane moving container {container_no} from vessel {vessel_id} at time {self.env.now:.2f}")
        yield self.env.timeout(MOVE_CONTAINER_TIME)
        
        print(f"Truck transporting container {container_no} from vessel {vessel_id} at time {self.env.now:.2f}")
        yield self.env.timeout(TRUCK_TRIP_ROUND_TIME)
        
        print(f"Truck returned after transporting container {container_no} from vessel {vessel_id} at time {self.env.now:.2f}")
        self.trucks.release(trucks_req)
    

In [27]:
def vessel_generator(env):
    vessel = ContainerSimulation(env, AVAIL_BERTHS, AVAIL_CRANES, AVAIL_TRUCKS)
    
    vessel_id = 0
    while(True):
        yield env.timeout(random.expovariate(1 / VESSEL_AVG_ARRIVAL_INTERVAL))
        vessel_id += 1
        env.process(vessel.process_for_unloading_containers(vessel_id))

In [28]:
env = simpy.Environment()
env.process(vessel_generator(env))
env.run(SIMULATION_TIME)

Vessel 1 arriving at time 296.55
Vessel 1 berthing at time 296.55
Quay crane moving container 1 from vessel 1 at time 296.55
Truck transporting container 1 from vessel 1 at time 299.55
Truck returned after transporting container 1 from vessel 1 at time 305.55
Quay crane moving container 2 from vessel 1 at time 305.55
Truck transporting container 2 from vessel 1 at time 308.55
Truck returned after transporting container 2 from vessel 1 at time 314.55
Quay crane moving container 3 from vessel 1 at time 314.55
Truck transporting container 3 from vessel 1 at time 317.55
Truck returned after transporting container 3 from vessel 1 at time 323.55
Quay crane moving container 4 from vessel 1 at time 323.55
Vessel 2 arriving at time 326.41
Vessel 2 berthing at time 326.41
Quay crane moving container 1 from vessel 2 at time 326.41
Truck transporting container 4 from vessel 1 at time 326.55
Truck transporting container 1 from vessel 2 at time 329.41
Truck returned after transporting container 4 fr

### References
- &emsp; &emsp; [https://simpy.readthedocs.io/en/latest/]
- &emsp; &emsp; [https://simpy.readthedocs.io/en/latest/examples/bank_renege.html]
- &emsp; &emsp; [https://docs.python.org/3/library/random.html#random.expovariate]
- &emsp; &emsp; [https://en.wikipedia.org/wiki/Terminal_tractor]
- &emsp; &emsp; [https://simpy.readthedocs.io/en/latest/api_reference/simpy.core.html#simpy.core.Environment.now]